In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from caveclient import CAVEclient

import sys
sys.path.append('utils')
from FANC_utils import filter_syn_df, hl_nt_dict

In [2]:
# Initialize CAVEclient
# This step requires an authentication token, see https://caveclient.readthedocs.io/en/latest/guide/authentication.html
client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.materialize.version = 840

In [18]:
# Load table 

# Select claw or hook neurons
cell_type = ['claw_flx','claw_ext']
#cell_type = ['hook_flx','hook_ext']

df = client.materialize.query_table('feco_axons_v0', 
                                    filter_in_dict = {'cell_type': cell_type})

# Exclude partially reconstructed claw axons
if 'claw_flx' in cell_type:
    pt_position_to_exclude = np.array([[9093, 101264, 3740], [9401, 100947, 3740]])
    ix_to_exclude = []
    for ix, i in df.iterrows():
        if (df.pt_position[ix]==pt_position_to_exclude).all(axis=1).any():
            ix_to_exclude.append(ix)
    df = df.drop(ix_to_exclude)

In [19]:
# Get IDs of FeCO neurons
feco_ids = df.pt_root_id.tolist()

In [20]:
# Query input synapses 
syn_df = client.materialize.synapse_query(post_ids = feco_ids)

In [21]:
# Remove connections less than syn_thresh from syn_df
syn_thresh = 3
syn_df = filter_syn_df(syn_df, syn_thresh)

In [22]:
# Add column syn_count
syn_df['syn_count'] = 1

In [23]:
# Sum synapses for each partner neuron
syn_df_sum = syn_df.groupby(['pre_pt_root_id'])['syn_count'].sum().reset_index().sort_values('syn_count', ascending=False)

In [24]:
# Get IDs of partner neurons
partner_ids = syn_df_sum.pre_pt_root_id.unique().tolist()

In [25]:
# Load annotation table with hemilineages of partner neurons 
partner_df = client.materialize.query_table('upstream_feco_axons')
partner_df_ids = partner_df.pt_root_id.tolist()

In [26]:
# Determine number of cholinergic (ACh), GABAergic (GABA), and glutamatergic (Glu) synapses 
n_syn_nt = {}
n_syn_nt['ACh'] = 0
n_syn_nt['GABA'] = 0
n_syn_nt['Glu'] = 0
n_syn_nt['Unknown'] = 0

nt_dict = hl_nt_dict()
nt_dict['hook'] = 'ACh'
nt_dict['club'] = 'ACh'
nt_dict['hair_plate'] = 'ACh'
nt_dict['descending'] = 'Unknown'

for i in partner_ids:
    # Get number of synapses
    n_syn = syn_df_sum.syn_count[syn_df_sum.pre_pt_root_id == i].item()
    # Get hemilineage of partner neuron
    if i in partner_df_ids:
        cell_type = partner_df.cell_type[partner_df.pt_root_id == i].item()
        # Look up primary neurotransmitter of hemilineage 
        nt = nt_dict[cell_type]
    else: 
        nt = 'Unknown'

    if nt in ['ACh', 'GABA', 'Glu']:
        n_syn_nt[nt] += n_syn
    else:
        n_syn_nt['Unknown'] += n_syn

In [27]:
n_syn_nt

{'ACh': 3, 'GABA': 230, 'Glu': 0, 'Unknown': 0}

In [29]:
230/233

0.9871244635193133